In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint

In [3]:
with open("hazes.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()
print("Loaded a corpus of {0} characters".format(len(corpus)))

Loaded a corpus of 41701 characters


In [4]:
# Get a unique identifier for each char in the corpus, then make some dicts to ease encoding and decoding
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

print(chars)
print("Our corpus contains {0} unique characters.".format(num_chars))
print(encoding)

['\n', ' ', '"', "'", '(', ')', ',', '.', '1', '2', ':', ';', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Our corpus contains 39 unique characters.
{'\n': 0, ' ': 1, '"': 2, "'": 3, '(': 4, ')': 5, ',': 6, '.': 7, '1': 8, '2': 9, ':': 10, ';': 11, '?': 12, '`': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'f': 19, 'g': 20, 'h': 21, 'i': 22, 'j': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}


In [5]:
# it slices, it dices, it makes julienned datasets!
# chop up our data into X and y, slice into roughly (num_chars / skip) overlapping 'sentences'
# of length sentence_length, and encode the chars
sentence_length = 20
skip = 1
X_data = []
y_data = []

for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])

In [6]:
print(len(X_data))

41681


In [7]:
X_data[1], y_data[1]

([3, 27, 1, 38, 28, 28, 27, 1, 35, 14, 31, 1, 20, 22, 31, 32, 18, 30, 18, 27],
 1)

In [8]:
X_data[2], y_data[2]

([27, 1, 38, 28, 28, 27, 1, 35, 14, 31, 1, 20, 22, 31, 32, 18, 30, 18, 27, 1],
 23)

In [9]:
num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}".format(num_sentences, sentence_length))

Sliced our corpus into 41681 sentences of length 20


In [10]:
print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=np.bool)
y = np.zeros((num_sentences, num_chars), dtype=np.bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Vectorizing X and y...


In [11]:
X[1]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, F

In [12]:
y[1]

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [13]:
# Double check our vectorized data before we sink hours into fitting a model
print("Sanity check y. Dimension: {0} # Sentences: {1} Characters in corpus: {2}".format(y.shape, num_sentences, len(chars)))
print("Sanity check X. Dimension: {0} Sentence length: {1}".format(X.shape, sentence_length))

Sanity check y. Dimension: (41681, 39) # Sentences: 41681 Characters in corpus: 39
Sanity check X. Dimension: (41681, 20, 39) Sentence length: 20


In [14]:
# Define our model
print("Let's build model 1")
model = Sequential()
model.add(LSTM(128, input_shape=(sentence_length, num_chars)))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary() 

Let's build model 1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               86016     
_________________________________________________________________
dense (Dense)                (None, 39)                5031      
_________________________________________________________________
activation (Activation)      (None, 39)                0         
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Dump our model architecture to a file so we can load it elsewhere
# Find out how to load a model? ,
# return_sequences=True
architecture = model.to_yaml()
with open('model.yaml', 'a') as model_file:
    model_file.write(architecture)

# Set up checkpoints, and save trained model
file_path="weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks = [checkpoint]

# Find out how to load the trained checkpoint?
# Lets go, action time!
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks)

Epoch 1/20
326/326 [==============================] - 26s 68ms/step - loss: 2.9864

Epoch 00001: loss improved from inf to 2.77779, saving model to weights-01.hdf5
Epoch 2/20
326/326 [==============================] - 23s 70ms/step - loss: 2.3016

Epoch 00002: loss improved from 2.77779 to 2.22553, saving model to weights-02.hdf5
Epoch 3/20
326/326 [==============================] - 22s 69ms/step - loss: 2.0563

Epoch 00003: loss improved from 2.22553 to 2.02973, saving model to weights-03.hdf5
Epoch 4/20
326/326 [==============================] - 24s 74ms/step - loss: 1.9543

Epoch 00004: loss improved from 2.02973 to 1.92984, saving model to weights-04.hdf5
Epoch 5/20
326/326 [==============================] - 23s 71ms/step - loss: 1.8635

Epoch 00005: loss improved from 1.92984 to 1.85900, saving model to weights-05.hdf5
Epoch 6/20
326/326 [==============================] - 24s 73ms/step - loss: 1.8008

Epoch 00006: loss improved from 1.85900 to 1.79317, saving model to weights-06.h

In [ ]:
#model.add(GRU(256),return_sequences=True)
#model.add(GRU(256))